In [2]:
%matplotlib notebook
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import seaborn as sns
import calendar
import random
import json 
import os

In [4]:
files = os.listdir('Data/history') 
df = pd.DataFrame()
for f in files:
    df1 = pd.read_csv(f'Data/history/{f}')
    df = pd.concat([df1, df], axis=0, ignore_index=True) 

#df = df.loc[ df['Description'] == 'ROPA CABALLERO R MAYA' ]
df.head(2)

,Transaction Date,Post Date,Description,Category,Type,Amount,Memo
0,1/19/2021,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,NaN
1,1/19/2021,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,NaN


In [5]:
df.drop_duplicates(subset=['Transaction Date','Description','Type','Amount'], keep="first", inplace=True)
df.head(2)

,Transaction Date,Post Date,Description,Category,Type,Amount,Memo
0,1/19/2021,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,NaN
3,1/18/2021,1/20/2021,MOMIJI RESTAURANT.,Food & Drink,Sale,-112.36,NaN


## Read in Dirty CSV

In [6]:
# df_dirty = pd.read_csv('Data/CC_2020YTD_Dirty.csv')

In [7]:
df_dirty=df
df_dirty

,Transaction Date,Post Date,Description,Category,Type,Amount,Memo
0,1/19/2021,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,NaN
3,1/18/2021,1/20/2021,MOMIJI RESTAURANT.,Food & Drink,Sale,-112.36,NaN
4,1/17/2021,1/19/2021,PENN QUARTER SPORTS TA,Food & Drink,Sale,-21.00,NaN
5,1/18/2021,1/19/2021,Prime Video*7J5P50IU3,Shopping,Sale,-6.35,NaN
6,1/16/2021,1/18/2021,SAFEWAY 2737,Groceries,Sale,-37.21,NaN
...,...,...,...,...,...,...,...
1174,01/02/2020,01/03/2020,WALMART.COM,Shopping,Sale,-52.50,NaN
1175,01/02/2020,01/03/2020,UBER TRIP,Travel,Sale,-8.00,NaN
1176,01/01/2020,01/02/2020,UBER TRIP,Travel,Sale,-13.09,NaN
1177,12/31/2019,01/02/2020,7-ELEVEN 33452,Gas,Sale,-6.20,NaN


## Clean Dirty CSV

In [8]:
df_clean = df_dirty.copy()  

In [9]:
#Split payments and Expenses from the Amount column
df = df_clean.copy()  
df['Expenses'] = df['Amount'] 
df['Payments'] = df['Amount']  
df['Expenses'] = df.loc[df['Expenses'] < 0, 'Expenses']
df['Payments'] = df.loc[df['Payments'] > 0, 'Payments']
#drop memo column
df.drop(['Memo'], axis=1, inplace=True)
#

df

,Transaction Date,Post Date,Description,Category,Type,Amount,Expenses,Payments
0,1/19/2021,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,-0.85,NaN
3,1/18/2021,1/20/2021,MOMIJI RESTAURANT.,Food & Drink,Sale,-112.36,-112.36,NaN
4,1/17/2021,1/19/2021,PENN QUARTER SPORTS TA,Food & Drink,Sale,-21.00,-21.00,NaN
5,1/18/2021,1/19/2021,Prime Video*7J5P50IU3,Shopping,Sale,-6.35,-6.35,NaN
6,1/16/2021,1/18/2021,SAFEWAY 2737,Groceries,Sale,-37.21,-37.21,NaN
...,...,...,...,...,...,...,...,...
1174,01/02/2020,01/03/2020,WALMART.COM,Shopping,Sale,-52.50,-52.50,NaN
1175,01/02/2020,01/03/2020,UBER TRIP,Travel,Sale,-8.00,-8.00,NaN
1176,01/01/2020,01/02/2020,UBER TRIP,Travel,Sale,-13.09,-13.09,NaN
1177,12/31/2019,01/02/2020,7-ELEVEN 33452,Gas,Sale,-6.20,-6.20,NaN


## Save Clean to new CSV

In [10]:
# create new empty column: Day  
df['Day'] = ''
   
# create new empty column: Month  
df['Month'] = ''
df
# create new empty column: Year
df['Year'] = ''
df
# **https://www.dataindependent.com/pandas/pandas-to-datetime/
# Set new column Day = the day extracted from Tranaction Date column
df['Day'] = df['Transaction Date'].apply(lambda x: x.split('/')[1])
df
# Set new column Month = the month extracted from Tranaction Date column
df['Month'] = df['Transaction Date'].apply(lambda x: x.split('/')[0])
df
# Set new column Year = the month extracted from Tranaction Date column
df['Year'] = df['Transaction Date'].apply(lambda x: x.split('/')[2])
df 
# Reorder Columns
df = df.reindex(columns= ['Transaction Date', 'Year', 'Month', 'Day', 'Post Date', 'Description', 'Category', 'Type', 'Amount', 'Expenses', 'Payments'])
# Rename column
df = df.rename(columns ={'Transaction Date':'TransactionDate', 'Post Date': 'PostDate'})

df_clean = df
df

,TransactionDate,Year,Month,Day,PostDate,Description,Category,Type,Amount,Expenses,Payments
0,1/19/2021,2021,1,19,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,-0.85,NaN
3,1/18/2021,2021,1,18,1/20/2021,MOMIJI RESTAURANT.,Food & Drink,Sale,-112.36,-112.36,NaN
4,1/17/2021,2021,1,17,1/19/2021,PENN QUARTER SPORTS TA,Food & Drink,Sale,-21.00,-21.00,NaN
5,1/18/2021,2021,1,18,1/19/2021,Prime Video*7J5P50IU3,Shopping,Sale,-6.35,-6.35,NaN
6,1/16/2021,2021,1,16,1/18/2021,SAFEWAY 2737,Groceries,Sale,-37.21,-37.21,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1174,01/02/2020,2020,01,02,01/03/2020,WALMART.COM,Shopping,Sale,-52.50,-52.50,NaN
1175,01/02/2020,2020,01,02,01/03/2020,UBER TRIP,Travel,Sale,-8.00,-8.00,NaN
1176,01/01/2020,2020,01,01,01/02/2020,UBER TRIP,Travel,Sale,-13.09,-13.09,NaN
1177,12/31/2019,2019,12,31,01/02/2020,7-ELEVEN 33452,Gas,Sale,-6.20,-6.20,NaN


In [11]:
df_clean.to_csv('Data/CC_2020YTD_Clean.csv', index=False)

## Visualize the CLean

In [12]:
df_clean = pd.read_csv('Data/CC_2020YTD_Clean.csv')

In [13]:
df = df_clean
df

,TransactionDate,Year,Month,Day,PostDate,Description,Category,Type,Amount,Expenses,Payments
0,1/19/2021,2021,1,19,1/20/2021,Zipcar Toll JAN 07,Travel,Sale,-0.85,-0.85,NaN
1,1/18/2021,2021,1,18,1/20/2021,MOMIJI RESTAURANT.,Food & Drink,Sale,-112.36,-112.36,NaN
2,1/17/2021,2021,1,17,1/19/2021,PENN QUARTER SPORTS TA,Food & Drink,Sale,-21.00,-21.00,NaN
3,1/18/2021,2021,1,18,1/19/2021,Prime Video*7J5P50IU3,Shopping,Sale,-6.35,-6.35,NaN
4,1/16/2021,2021,1,16,1/18/2021,SAFEWAY 2737,Groceries,Sale,-37.21,-37.21,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1103,01/02/2020,2020,1,2,01/03/2020,WALMART.COM,Shopping,Sale,-52.50,-52.50,NaN
1104,01/02/2020,2020,1,2,01/03/2020,UBER TRIP,Travel,Sale,-8.00,-8.00,NaN
1105,01/01/2020,2020,1,1,01/02/2020,UBER TRIP,Travel,Sale,-13.09,-13.09,NaN
1106,12/31/2019,2019,12,31,01/02/2020,7-ELEVEN 33452,Gas,Sale,-6.20,-6.20,NaN


In [ ]:
df.to_json('Data/df_Clean.js', orient='records')
with open('Data/df_Clean.js') as datafile:
    data = json.load(datafile)
    data = str(data)
    data = 'var data = ' + data
    data = data.replace(': None', ': null')
    
#data

with open('Data/df_Clean.js', 'w') as datafile:
    datafile.write(data)

In [15]:
df.to_json('Static/data/df_Clean.json', orient='records')
